In [ ]:
from sequential_tasks import EchoData
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# Specify experiment settings and prepare the data

In [ ]:
# experiments settings
settings = {
    "series_length": 20000,
    "echo_step": 3,
    "truncated_length": 20,
    "batch_size": 5,
    "h_units": 4,
    "max_epochs": 5
}

In [ ]:
#training data
train_data_gen = EchoData(
        series_length=settings['series_length'],
        truncated_length=settings['truncated_length'],
        echo_step=settings['echo_step'],
        batch_size=settings['batch_size'])
train_size = len(train_data_gen)

#testing 
test_data_gen = EchoData(
        series_length=settings['series_length'],
        truncated_length=settings['truncated_length'],
        echo_step=settings['echo_step'],
        batch_size=settings['batch_size'])
test_size = len(test_data_gen)    

# Define the model

In [ ]:
class SimpleRNN(nn.Module):

    def __init__(self, input_size, rnn_hidden_size, output_size):

        super(SimpleRNN, self).__init__()
        self.rnn_hidden_size = rnn_hidden_size
        self.rnn = torch.nn.RNN(input_size, self.rnn_hidden_size, num_layers=1, nonlinearity='relu', batch_first=True)
        self.linear = torch.nn.Linear(rnn_hidden_size, 1)

    def forward(self, x, hidden):
        x, hidden = self.rnn(x, hidden)  
        x = self.linear(x)
        return nn.Sigmoid()(x), hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        return weight.new(1, batch_size, self.rnn_hidden_size).zero_()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Define training and test loops

In [ ]:
def train(hidden):
    model.train()
       
    correct = 0
    for batch_idx in range(train_size):
        data, target = train_data_gen[batch_idx]
        data, target = torch.from_numpy(data).float().to(device), torch.from_numpy(target).float().to(device)
        optimizer.zero_grad()
        y_pred, hidden = model(data, hidden)
        loss = criterion(y_pred, target)
        loss.backward(retain_graph=True)
        optimizer.step()
        
        pred = (y_pred > 0.5).float()
        correct += (pred == target).sum().item()
        
    return correct, loss, hidden    

In [ ]:
def test(hidden):
    model.eval()   
    correct = 0
    with torch.no_grad():
        for batch_idx in range(test_size):
            data, target = test_data_gen[batch_idx]
            data, target = torch.from_numpy(data).float().to(device), torch.from_numpy(target).float().to(device)
            y_pred, hidden = model(data, hidden)
            
            pred = (y_pred > 0.5).float()
            correct += (pred == target).sum().item()

    return correct

# Initialize the Model and Optimizer

In [ ]:
feature_dim = 1 #since we have a scalar series
model = SimpleRNN(1, settings['h_units'], 1)  
model.to(device)
hidden = model.init_hidden(train_data_gen.batch_size) #initialize hidden states for RNN  
        
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

In [ ]:
epochs = settings['max_epochs']
epoch = 0

while epoch < epochs:
    correct, loss, hidden = train(hidden)
    epoch += 1
    train_accuracy = float(correct) / train_size
    print('Train Epoch: {}/{}, loss: {:.4f}, accuracy {:2.2f}'.format(epoch, epochs, loss.item(), train_accuracy))

#test    
correct = test(hidden)
test_accuracy = float(correct) / test_size
print('\nTest accuracy: {}'.format(test_accuracy))